In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, Input
from tensorflow.keras.utils import to_categorical

# Function to load and preprocess a single CSV file
def load_and_preprocess_csv(file_path):
    data = pd.read_csv(file_path)
    data['count'] = data.groupby('slice_index')['count'].transform(lambda x: x / x.sum())
    pivot_data = data.pivot(index='slice_index', columns='bin_start', values='count').fillna(0)
    return pivot_data

# Function to pad arrays to the same shape
def pad_array(array, max_shape):
    pad_width = [(0, max_shape[i] - array.shape[i]) for i in range(len(array.shape))]
    padded_array = np.pad(array, pad_width, mode='constant', constant_values=0)
    return padded_array

# Function to load data from a directory
def load_data_from_directory(directory):
    positive_files = sorted([f for f in os.listdir(directory) if f.startswith('positive')])
    negative_files = sorted([f for f in os.listdir(directory) if f.startswith('negative')])
    
    signals = []
    labels = []
    max_shape = (0, 0)

    # First pass to determine the maximum shape
    for pos_file, neg_file in zip(positive_files, negative_files):
        pos_index = int(pos_file.split('_')[1].split('.')[0])
        neg_index = int(neg_file.split('_')[1].split('.')[0])
        if pos_index == neg_index:
            pos_data = load_and_preprocess_csv(os.path.join(directory, pos_file))
            neg_data = load_and_preprocess_csv(os.path.join(directory, neg_file))
            combined_data = np.concatenate((pos_data, neg_data), axis=1)
            max_shape = (max(max_shape[0], combined_data.shape[0]), max(max_shape[1], combined_data.shape[1]))

    # Second pass to pad arrays to the maximum shape
    for pos_file, neg_file in zip(positive_files, negative_files):
        pos_index = int(pos_file.split('_')[1].split('.')[0])
        neg_index = int(neg_file.split('_')[1].split('.')[0])
        if pos_index == neg_index:
            pos_data = load_and_preprocess_csv(os.path.join(directory, pos_file))
            neg_data = load_and_preprocess_csv(os.path.join(directory, neg_file))
            combined_data = np.concatenate((pos_data, neg_data), axis=1)
            padded_data = pad_array(combined_data, max_shape)
            signals.append(padded_data)
            labels.append(1 if 'Glitch_volumes' in directory else 0)
    
    return np.array(signals), np.array(labels)

# Load your data
merger_directory = '/home/arutkeerthi/Downloads/Glitchveto/Merger_volumes/1200MPC-2-Noisy'
glitch_directory = '/home/arutkeerthi/Downloads/Glitchveto/Glitch_volumes'

X_merger, y_merger = load_data_from_directory(merger_directory)
X_glitch, y_glitch = load_data_from_directory(glitch_directory)

# Combine merger and glitch data
X = np.concatenate((X_merger, X_glitch), axis=0)
y = np.concatenate((y_merger, y_glitch), axis=0)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert labels to categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Define the CNN model
input_shape = (X_train.shape[1], X_train.shape[2])
inputs = Input(shape=input_shape)

x = Conv1D(filters=32, kernel_size=3, activation='relu')(inputs)
x = MaxPooling1D(pool_size=2)(x)
x = Dropout(0.25)(x)

x = Conv1D(filters=64, kernel_size=3, activation='relu')(x)
x = MaxPooling1D(pool_size=2)(x)
x = Dropout(0.25)(x)

x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)

outputs = Dense(2, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy}')

2024-08-01 11:20:38.877002: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-01 11:20:39.038627: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-01 11:20:39.040022: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-01 11:20:40.017997: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-08-01 11:21:10.153082: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Do

Epoch 1/20


2024-08-01 11:21:11.449297: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 396519900 exceeds 10% of free system memory.


 3/19 [===>..........................] - ETA: 0s - loss: 0.6690 - accuracy: 0.7292 

2024-08-01 11:21:12.489170: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 21325440 exceeds 10% of free system memory.
2024-08-01 11:21:12.489270: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 21325440 exceeds 10% of free system memory.
2024-08-01 11:21:12.489329: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 21325440 exceeds 10% of free system memory.
2024-08-01 11:21:12.489389: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 21325440 exceeds 10% of free system memory.


19/19 [==============================] - 2s 55ms/step - loss: 0.3650 - accuracy: 0.8773 - val_loss: 0.1170 - val_accuracy: 0.8926
Epoch 2/20
19/19 [==============================] - 1s 36ms/step - loss: 0.0754 - accuracy: 0.9361 - val_loss: 0.0534 - val_accuracy: 1.0000
Epoch 3/20
19/19 [==============================] - 1s 37ms/step - loss: 0.0204 - accuracy: 1.0000 - val_loss: 0.0034 - val_accuracy: 1.0000
Epoch 4/20
19/19 [==============================] - 1s 36ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 1.5532e-04 - val_accuracy: 1.0000
Epoch 5/20
19/19 [==============================] - 1s 36ms/step - loss: 1.4512e-04 - accuracy: 1.0000 - val_loss: 7.1122e-05 - val_accuracy: 1.0000
Epoch 6/20
19/19 [==============================] - 1s 37ms/step - loss: 1.0328e-04 - accuracy: 1.0000 - val_loss: 5.4176e-05 - val_accuracy: 1.0000
Epoch 7/20
19/19 [==============================] - 1s 38ms/step - loss: 1.0370e-04 - accuracy: 1.0000 - val_loss: 4.4751e-05 - val_accuracy: 1.0